# BART Fine-tuning Script Development


We will formulate parallel Text Style Transfer as Conditional Generation task and fine-tune the `bart-base` pre-trained model on the Wiki Neutrality Corpus in similar fashion to a Summarization use case. More detail on the modeling approach can be [found here](https://www.notion.so/fastforwardlabs/Modeling-Approach-16576e450a2f4237b595acd926d245f5).

In this notebook, we intend to modify the exemplar [Huggingface summarization fine-tuning script](https://github.com/huggingface/transformers/tree/master/examples/pytorch/summarization) to fit our use case, model, dataset, and project scope.

**NOTE -** 
- In order to run the latest version of the example scripts from Hugggingface, you **must install the library from source and install some example-specific requirements**. Details [found here](https://github.com/huggingface/transformers/tree/master/examples#important-note).
- In order to get the `scripts/train_job.py` to run as a CML Job, I had to manually run `pip3 install attrs -t /home/cdsw/.local/lib/python3.9/site-packages --no-user`. This allowed me to force an install of the newer attrs package into the .local site-packages, which is needed because runtimes come with an older version (attr) that lives in the /usr/lib directory and gets picked up by default -- throwing an error.

  

### TO DO
- need to add automated script that installs HF from source by first cloning the proper tag (as documented in link above)

## Training Config

**Default `Trainer` Configs:**
- n_epochs: 3
- 

In [ ]:
import time
time.sleep(60*60*10)

In [18]:
watch -n1 nvidia-smi

SyntaxError: invalid syntax (<ipython-input-18-1a47b2a41d94>, line 1)

ERROR:root:Cannot parse: 1:10: watch -n1 nvidia-smi
Traceback (most recent call last):
  File "/home/cdsw/.local/lib/python3.9/site-packages/lab_black.py", line 218, in format_cell
    formatted_code = _format_code(cell)
  File "/home/cdsw/.local/lib/python3.9/site-packages/lab_black.py", line 29, in _format_code
    return format_str(src_contents=code, mode=FileMode())
  File "src/black/__init__.py", line 1131, in format_str
  File "src/black/__init__.py", line 1141, in _format_str_once
  File "src/black/parsing.py", line 128, in lib2to3_parse
black.parsing.InvalidInput: Cannot parse: 1:10: watch -n1 nvidia-smi


In [ ]:
python3 scripts/run_TST.py \
    --model_name_or_path facebook/bart-base \
    --do_train \
    --do_eval \
    --do_predict \
    --dataset_name "wnc_one_word" \
    --output_dir "./models/bart-tst-oneword" \
    --overwrite_output_dir \
    --evaluation_strategy "steps" \
    --logging_strategy "steps" \
    --per_device_train_batch_size=4 \
    --per_device_eval_batch_size=4 \
    --predict_with_generate \
    --load_best_model_at_end=True \
    --max_train_samples=1000

## Load Fine-Tuned Model

In [1]:
import os
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq
from datasets import load_from_disk, load_metric

%load_ext lab_black
%load_ext tensorboard

In [2]:
DATASETS_PATH = "/home/cdsw/data/processed/WNC_oneword"
MODEL_PATH = "/home/cdsw/models/bart-tst-oneword-5epoch"

wnc_datasets = load_from_disk(DATASETS_PATH)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH)

## Evaluation Walkthrough

In [3]:
examples = wnc_datasets["test"][:20]
examples

{'rev_id': ['318427508',
  '235640083',
  '37561168',
  '101665256',
  '480248865',
  '317239242',
  '264200064',
  '194952441',
  '66405411',
  '14497516',
  '275953052',
  '701211560',
  '747113529',
  '305605305',
  '360725575',
  '37134136',
  '58108525',
  '515090728',
  '857485368',
  '789013656'],
 'source_text': ['in april 2009 a brazilian human rights group, torture never again, awarded the five its chico mendes medal, because their rights had been violated.',
  'the 51 day standoff and ensuing murder of 76 men, women, and children--the branch davidians--in waco, texas.',
  'mark oaten (born 8 march 1964, watford) is a disgraced liberal democrat politician in the united kingdom, and member of parliament for the winchester constituency.',
  'another infamous period of colonisation in ancient times was from the romans.',
  'photo sequence of astonishing 2005 chicagoland crash with ryan briscoe.',
  'jesus of nazareth is probably mentioned in two passages of the work the antiquit

In [4]:
max_source_length = 1024
max_target_length = 128


def preprocess_function(examples: dict):

    inputs = examples["source_text"]
    targets = examples["target_text"]

    model_inputs = tokenizer(
        inputs,
        max_length=max_source_length,
        padding=True,
        truncation=True,
        return_tensors="pt",
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=max_target_length,
            padding=True,
            truncation=True,
            return_tensors="pt",
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [5]:
model_inputs = preprocess_function(examples)

In [6]:
model_inputs["input_ids"].shape

torch.Size([20, 50])

### Generate Text

In [7]:
def generate_text(examples: dict):

    model_inputs = preprocess_function(examples)

    outputs = model.generate(
        model_inputs["input_ids"],
        max_length=max_target_length,
        min_length=4,
        length_penalty=2,
        num_beams=4,
        early_stopping=True,
    )

    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [8]:
preds = generate_text(examples)
preds

['in april 2009 a brazilian human rights group, torture never again, awarded the five its chico mendes medal, claiming that their rights had been violated.',
 'the 51 day standoff and ensuing killing of 76 men, women, and children--the branch davidians--in waco, texas.',
 'mark oaten (born 8 march 1964, watford) is a liberal democrat politician in the united kingdom, and member of parliament for the winchester constituency.',
 'another period of colonisation in ancient times was from the romans.',
 'photo sequence of 2005 chicagoland crash with ryan briscoe.',
 'jesus of nazareth is mentioned in two passages of the work the antiquities of the jews by the jewish historian josephus, written in the late first century ad.',
 'israeli aims to reduce gazan civilian casualties',
 'his 45-year career exceeded that of any other studio head.',
 'they see the calling of such councils, for example, by a roman emperor lacking the divine authority as preposterous and assert that the emperors used th

In [9]:
labels = examples["target_text"]
labels

['in april 2009 a brazilian human rights group, torture never again, awarded the five its chico mendes medal, under the pretext that their rights had been violated.',
 'the 51 day standoff and ensuing deaths of 76 men, women, and children--the branch davidians--in waco, texas.',
 'mark oaten (born 8 march 1964, watford) is a liberal democrat politician in the united kingdom, and member of parliament for the winchester constituency.',
 'another period of colonisation in ancient times was from the romans.',
 'photo sequence of 2005 chicagoland crash with ryan briscoe.',
 'jesus of nazareth is possibly mentioned in two passages of the work the antiquities of the jews by the jewish historian josephus, written in the late first century ad.',
 'israeli efforts to reduce gazan civilian casualties',
 'his 45-year career was longer than that of any other studio head.',
 'they see the calling of such councils, for example, by a roman emperor lacking the divine authority as groundless and assert 

### Evaluate

In [210]:
import nltk

In [211]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    # # rougeLSum expects newline after each sentence
    # preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    # labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels

In [212]:
preds, labels = postprocess_text(preds, labels)

#### ROUGE

In [153]:
metric = load_metric("rouge")

In [155]:
result = metric.compute(predictions=preds, references=labels, use_stemmer=True)
result

{'rouge1': AggregateScore(low=Score(precision=0.9523809523809523, recall=0.9, fmeasure=0.9473684210526316), mid=Score(precision=0.9761904761904762, recall=0.9261904761904762, fmeasure=0.949874686716792), high=Score(precision=1.0, recall=0.9523809523809523, fmeasure=0.9523809523809523)),
 'rouge2': AggregateScore(low=Score(precision=0.875, recall=0.7777777777777778, fmeasure=0.823529411764706), mid=Score(precision=0.9125, recall=0.8638888888888889, fmeasure=0.886764705882353), high=Score(precision=0.95, recall=0.95, fmeasure=0.9500000000000001)),
 'rougeL': AggregateScore(low=Score(precision=0.9523809523809523, recall=0.9, fmeasure=0.9473684210526316), mid=Score(precision=0.9761904761904762, recall=0.9261904761904762, fmeasure=0.949874686716792), high=Score(precision=1.0, recall=0.9523809523809523, fmeasure=0.9523809523809523)),
 'rougeLsum': AggregateScore(low=Score(precision=0.9523809523809523, recall=0.9, fmeasure=0.9473684210526316), mid=Score(precision=0.9761904761904762, recall=0.

In [156]:
{key: value.mid.fmeasure * 100 for key, value in result.items()}

{'rouge1': 94.9874686716792,
 'rouge2': 88.6764705882353,
 'rougeL': 94.9874686716792,
 'rougeLsum': 94.9874686716792}

#### BLEU

In [213]:
preds

['in april 2009 a brazilian human rights group, torture never again, awarded the five its chico mendes medal, because their rights had been violated.',
 'the 51 day standoff and ensuing murder of 76 men, women, and children--the branch davidians--in waco, texas.',
 'mark oaten (born 8 march 1964, watford) is a liberal democrat politician in the united kingdom, and member of parliament for the winchester constituency.',
 'another period of colonisation in ancient times was from the romans.',
 'photo sequence of 2005 chicagoland crash with ryan briscoe.',
 'jesus of nazareth is probably mentioned in two passages of the work the antiquities of the jews by the jewish historian josephus, written in the late first century ad.',
 'israeli attempts to reduce gazan civilian casualties',
 'his 45-year career exceeded that of any other studio head.',
 'they see the calling of such councils, for example, by a roman emperor lacking the divine authority as preposterous and assert that the emperors u

In [214]:
labels

[['in april 2009 a brazilian human rights group, torture never again, awarded the five its chico mendes medal, under the pretext that their rights had been violated.'],
 ['the 51 day standoff and ensuing deaths of 76 men, women, and children--the branch davidians--in waco, texas.'],
 ['mark oaten (born 8 march 1964, watford) is a liberal democrat politician in the united kingdom, and member of parliament for the winchester constituency.'],
 ['another period of colonisation in ancient times was from the romans.'],
 ['photo sequence of 2005 chicagoland crash with ryan briscoe.'],
 ['jesus of nazareth is possibly mentioned in two passages of the work the antiquities of the jews by the jewish historian josephus, written in the late first century ad.'],
 ['israeli efforts to reduce gazan civilian casualties'],
 ['his 45-year career was longer than that of any other studio head.'],
 ['they see the calling of such councils, for example, by a roman emperor lacking the divine authority as groun

In [198]:
preds = ["I am the same.", "oh yea nope"]
labels = [["I am the same."], ["oh yea nopee"]]

In [215]:
sum([(x == y[0]) for x, y in zip(preds, labels)]) / len(preds)

0.3

In [216]:
list(zip(preds, labels))[0]

('in april 2009 a brazilian human rights group, torture never again, awarded the five its chico mendes medal, because their rights had been violated.',
 ['in april 2009 a brazilian human rights group, torture never again, awarded the five its chico mendes medal, under the pretext that their rights had been violated.'])

In [187]:
import numpy as np

In [188]:
preds

['as of january 27, 2006, brokeback mountain had grossed over $46 million in north america and more than $10 million overseas.',
 'during the horseplay, hamlin fell and injured his hand.']

In [179]:
metric2 = load_metric("sacrebleu")

In [186]:
result2 = metric2.compute(predictions=preds, references=labels)

In [183]:
result2

{'score': 87.5854401903611,
 'counts': [36, 32, 29, 26],
 'totals': [37, 35, 33, 31],
 'precisions': [97.29729729729729,
  91.42857142857143,
  87.87878787878788,
  83.87096774193549],
 'bp': 0.9733349348192527,
 'sys_len': 37,
 'ref_len': 38}

### Inspect

In [10]:
examples["source_text"]

['in april 2009 a brazilian human rights group, torture never again, awarded the five its chico mendes medal, because their rights had been violated.',
 'the 51 day standoff and ensuing murder of 76 men, women, and children--the branch davidians--in waco, texas.',
 'mark oaten (born 8 march 1964, watford) is a disgraced liberal democrat politician in the united kingdom, and member of parliament for the winchester constituency.',
 'another infamous period of colonisation in ancient times was from the romans.',
 'photo sequence of astonishing 2005 chicagoland crash with ryan briscoe.',
 'jesus of nazareth is probably mentioned in two passages of the work the antiquities of the jews by the jewish historian josephus, written in the late first century ad.',
 'israeli attempts to reduce gazan civilian casualties',
 'his 45-year career exceeded that of any other studio head.',
 'they see the calling of such councils, for example, by a roman emperor lacking the divine authority as preposterous

In [13]:
wnc_datasets

DatasetDict({
    test: Dataset({
        features: ['rev_id', 'source_text', 'target_text'],
        num_rows: 967
    })
    train: Dataset({
        features: ['rev_id', 'source_text', 'target_text'],
        num_rows: 52093
    })
    validation: Dataset({
        features: ['rev_id', 'source_text', 'target_text'],
        num_rows: 671
    })
})

In [12]:
for source, target, pred in zip(examples["source_text"], labels, preds):
    print(f"SOURCE: {source} \n")
    print(f"TARGET: {target} \n")
    print(f"PREDIC: {pred} \n")
    print("-----------------------------------------------------------")

SOURCE: in april 2009 a brazilian human rights group, torture never again, awarded the five its chico mendes medal, because their rights had been violated. 

TARGET: in april 2009 a brazilian human rights group, torture never again, awarded the five its chico mendes medal, under the pretext that their rights had been violated. 

PREDIC: in april 2009 a brazilian human rights group, torture never again, awarded the five its chico mendes medal, claiming that their rights had been violated. 

-----------------------------------------------------------
SOURCE: the 51 day standoff and ensuing murder of 76 men, women, and children--the branch davidians--in waco, texas. 

TARGET: the 51 day standoff and ensuing deaths of 76 men, women, and children--the branch davidians--in waco, texas. 

PREDIC: the 51 day standoff and ensuing killing of 76 men, women, and children--the branch davidians--in waco, texas. 

-----------------------------------------------------------
SOURCE: mark oaten (born 8 

## Inference Walkthrough

In [91]:
sample_text = [
    "adequate testing for adverse health effects as well as performance data for these devices are seriously lacking.",
    "I am a really great human being",
]

# inputs = tokenizer.encode(
#     sample_text, return_tensors="pt", max_length=max_source_length, truncation=True
# )

inputs = tokenizer(sample_text, padding=True, truncation=True, return_tensors="pt")

In [96]:
inputs["input_ids"]

tensor([[    0, 37462,   877,  3044,    13, 12661,   474,  3038,    25,   157,
            25,   819,   414,    13,   209,  2110,    32,  3640, 12622,     4,
             2],
        [    0,   100,   524,    10,   269,   372,  1050,   145,     2,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1]])

In [98]:
outputs = model.generate(
    inputs["input_ids"],
    max_length=max_target_length,
    min_length=4,
    length_penalty=2,
    num_beams=4,
    early_stopping=True,
)

In [99]:
outputs

tensor([[    2,     0, 37462,   877,  3044,    13, 12661,   474,  3038,    25,
           157,    25,   819,   414,    13,   209,  2110,    32, 12622,     4,
             2],
        [    2,     0,   100,   524,    10,   269,   372,  1050,   145,     2,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1]])

In [100]:
outputs.shape

torch.Size([2, 21])

In [101]:
outputs[0].shape

torch.Size([21])

In [103]:
tokenizer.batch_decode(outputs, skip_special_tokens=True)

['adequate testing for adverse health effects as well as performance data for these devices are lacking.',
 'I am a really great human being']

In [ ]:
tokenizer.decode(

In [14]:
inputs = tokenizer(sample_text)
inputs

{'input_ids': [0, 37462, 877, 3044, 13, 12661, 474, 3038, 25, 157, 25, 819, 414, 13, 209, 2110, 32, 3640, 12622, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [9]:
inputs.input_ids

[0,
 37462,
 877,
 3044,
 13,
 12661,
 474,
 3038,
 25,
 157,
 25,
 819,
 414,
 13,
 209,
 2110,
 32,
 3640,
 12622,
 4,
 2]

In [16]:
tokenizer.tokenize(sample_text)

['adequ',
 'ate',
 'Ġtesting',
 'Ġfor',
 'Ġadverse',
 'Ġhealth',
 'Ġeffects',
 'Ġas',
 'Ġwell',
 'Ġas',
 'Ġperformance',
 'Ġdata',
 'Ġfor',
 'Ġthese',
 'Ġdevices',
 'Ġare',
 'Ġseriously',
 'Ġlacking',
 '.']

In [15]:
tokenizer.convert_ids_to_tokens(inputs.input_ids)

['<s>',
 'adequ',
 'ate',
 'Ġtesting',
 'Ġfor',
 'Ġadverse',
 'Ġhealth',
 'Ġeffects',
 'Ġas',
 'Ġwell',
 'Ġas',
 'Ġperformance',
 'Ġdata',
 'Ġfor',
 'Ġthese',
 'Ġdevices',
 'Ġare',
 'Ġseriously',
 'Ġlacking',
 '.',
 '</s>']

In [10]:
tokenizer.convert_ids_to_tokens(inputs.input_ids)

['<s>',
 'adequ',
 'ate',
 'Ġtesting',
 'Ġfor',
 'Ġadverse',
 'Ġhealth',
 'Ġeffects',
 'Ġas',
 'Ġwell',
 'Ġas',
 'Ġperformance',
 'Ġdata',
 'Ġfor',
 'Ġthese',
 'Ġdevices',
 'Ġare',
 'Ġseriously',
 'Ġlacking',
 '.',
 '</s>']

In [11]:
tokenizer

PreTrainedTokenizerFast(name_or_path='/home/cdsw/models/bart-tst-oneword', vocab_size=50265, model_max_len=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)})

In [ ]:
def preprocess_function(examples):
    
    

In [3]:
model.generate("another infamous period of colonisation in ancient times was from the romans.")

AttributeError: 'str' object has no attribute 'shape'

In [32]:
wnc_datasets["test"][3]

{'rev_id': '101665256',
 'source_text': 'another infamous period of colonisation in ancient times was from the romans.',
 'target_text': 'another period of colonisation in ancient times was from the romans.'}

In [33]:
test = "another infamous period of colonisation in ancient times was from the romans."

In [34]:
tokenizer(test)

{'input_ids': [0, 30303, 13358, 675, 9, 17735, 3258, 11, 8178, 498, 21, 31, 5, 14233, 1253, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [36]:
wnc_datasets

DatasetDict({
    test: Dataset({
        features: ['rev_id', 'source_text', 'target_text'],
        num_rows: 967
    })
    train: Dataset({
        features: ['rev_id', 'source_text', 'target_text'],
        num_rows: 52093
    })
    validation: Dataset({
        features: ['rev_id', 'source_text', 'target_text'],
        num_rows: 671
    })
})

In [19]:
torch.cuda.get_device_name()

'Tesla V100-PCIE-32GB'

In [ ]:
/home/cdsw/models/bart-tst-oneword/runs

In [11]:
PORT = os.getenv("CDSW_READONLY_PORT")

In [15]:
PORT

'8100'

In [17]:
%tensorboard --logdir=/home/cdsw/models/bart-tst-oneword/runs --port $CDSW_READONLY_PORT

ERROR: Failed to launch TensorBoard (exited with 2).
Contents of stderr:
TensorFlow installation not found - running with reduced feature set.
usage: tensorboard [-h] [--helpfull] [--logdir PATH] [--logdir_spec PATH_SPEC]
                   [--host ADDR] [--bind_all] [--port PORT]
                   [--reuse_port BOOL] [--load_fast {false,auto,true}]
                   [--extra_data_server_flags EXTRA_DATA_SERVER_FLAGS]
                   [--grpc_creds_type {local,ssl,ssl_dev}]
                   [--grpc_data_provider PORT] [--purge_orphaned_data BOOL]
                   [--db URI] [--db_import] [--inspect] [--version_tb]
                   [--tag TAG] [--event_file PATH] [--path_prefix PATH]
                   [--window_title TEXT] [--max_reload_threads COUNT]
                   [--reload_interval SECONDS] [--reload_task TYPE]
                   [--reload_multifile BOOL]
                   [--reload_multifile_inactive_secs SECONDS]
                   [--generic_data TYPE]
            

In [9]:
os.getenv("CDSW_READONLY_PORT")

'8100'

In [1]:
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Optional

import datasets
import nltk  # Here to have a nice missing dependency error message early on
import numpy as np
from datasets import load_dataset, load_metric, load_from_disk

import transformers
from filelock import FileLock
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    set_seed,
)
from transformers.file_utils import is_offline_mode
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version
from transformers.utils.versions import require_version

%load_ext lab_black

In [7]:
??Seq2SeqTrainingArguments

Init signature:
Seq2SeqTrainingArguments(
    output_dir: str,
    overwrite_output_dir: bool = False,
    do_train: bool = False,
    do_eval: bool = False,
    do_predict: bool = False,
    evaluation_strategy: transformers.trainer_utils.IntervalStrategy = 'no',
    prediction_loss_only: bool = False,
    per_device_train_batch_size: int = 8,
    per_device_eval_batch_size: int = 8,
    per_gpu_train_batch_size: Optional[int] = None,
    per_gpu_eval_batch_size: Optional[int] = None,
    gradient_accumulation_steps: int = 1,
    eval_accumulation_steps: Optional[int] = None,
    learning_rate: float = 5e-05,
    weight_decay: float = 0.0,
    adam_beta1: float = 0.9,
    adam_beta2: float = 0.999,
    adam_epsilon: float = 1e-08,
    max_grad_norm: float = 1.0,
    num_train_epochs: float = 3.0,
    max_steps: int = -1,
    lr_scheduler_type: transformers.trainer_utils.SchedulerType = 'linear',
    warmup_ratio: float = 0.0,
    warmup_steps: int = 0,
    log_level: Optional[str] = '

In [2]:
# Will error if the minimal version of Transformers is not installed. Remove at your own risks.
# check_min_version("4.18.0.dev0")

require_version(
    "datasets>=1.8.0",
    "To fix: pip install -r examples/pytorch/summarization/requirements.txt",
)

logger = logging.getLogger(__name__)

try:
    nltk.data.find("tokenizers/punkt")
except (LookupError, OSError):
    if is_offline_mode():
        raise LookupError(
            "Offline mode: run this script without TRANSFORMERS_OFFLINE first to download nltk data files"
        )
    with FileLock(".lock") as lock:
        nltk.download("punkt", quiet=True)

In [3]:
@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={
            "help": "Path to pretrained model or model identifier from huggingface.co/models"
        }
    )
    config_name: Optional[str] = field(
        default=None,
        metadata={
            "help": "Pretrained config name or path if not the same as model_name"
        },
    )
    tokenizer_name: Optional[str] = field(
        default=None,
        metadata={
            "help": "Pretrained tokenizer name or path if not the same as model_name"
        },
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={
            "help": "Where to store the pretrained models downloaded from huggingface.co"
        },
    )
    use_fast_tokenizer: bool = field(
        default=True,
        metadata={
            "help": "Whether to use one of the fast tokenizer (backed by the tokenizers library) or not."
        },
    )
    model_revision: str = field(
        default="main",
        metadata={
            "help": "The specific model version to use (can be a branch name, tag name or commit id)."
        },
    )
    use_auth_token: bool = field(
        default=False,
        metadata={
            "help": "Will use the token generated when running `transformers-cli login` (necessary to use this script "
            "with private models)."
        },
    )
    resize_position_embeddings: Optional[bool] = field(
        default=None,
        metadata={
            "help": "Whether to automatically resize the position embeddings if `max_source_length` exceeds "
            "the model's position embeddings."
        },
    )

In [4]:
@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    lang: str = field(default=None, metadata={"help": "Language id for summarization."})

    dataset_name: Optional[str] = field(
        default=None,
        metadata={"help": "The name of the dataset to use (via the datasets library)."},
    )
    dataset_config_name: Optional[str] = field(
        default=None,
        metadata={
            "help": "The configuration name of the dataset to use (via the datasets library)."
        },
    )
    text_column: Optional[str] = field(
        default=None,
        metadata={
            "help": "The name of the column in the datasets containing the full texts (for summarization)."
        },
    )
    summary_column: Optional[str] = field(
        default=None,
        metadata={
            "help": "The name of the column in the datasets containing the summaries (for summarization)."
        },
    )
    train_file: Optional[str] = field(
        default=None,
        metadata={"help": "The input training data file (a jsonlines or csv file)."},
    )
    validation_file: Optional[str] = field(
        default=None,
        metadata={
            "help": "An optional input evaluation data file to evaluate the metrics (rouge) on "
            "(a jsonlines or csv file)."
        },
    )
    test_file: Optional[str] = field(
        default=None,
        metadata={
            "help": "An optional input test data file to evaluate the metrics (rouge) on "
            "(a jsonlines or csv file)."
        },
    )
    overwrite_cache: bool = field(
        default=False,
        metadata={"help": "Overwrite the cached training and evaluation sets"},
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "The number of processes to use for the preprocessing."},
    )
    max_source_length: Optional[int] = field(
        default=1024,
        metadata={
            "help": "The maximum total input sequence length after tokenization. Sequences longer "
            "than this will be truncated, sequences shorter will be padded."
        },
    )
    max_target_length: Optional[int] = field(
        default=128,
        metadata={
            "help": "The maximum total sequence length for target text after tokenization. Sequences longer "
            "than this will be truncated, sequences shorter will be padded."
        },
    )
    val_max_target_length: Optional[int] = field(
        default=None,
        metadata={
            "help": "The maximum total sequence length for validation target text after tokenization. Sequences longer "
            "than this will be truncated, sequences shorter will be padded. Will default to `max_target_length`."
            "This argument is also used to override the ``max_length`` param of ``model.generate``, which is used "
            "during ``evaluate`` and ``predict``."
        },
    )
    pad_to_max_length: bool = field(
        default=False,
        metadata={
            "help": "Whether to pad all samples to model maximum sentence length. "
            "If False, will pad the samples dynamically when batching to the maximum length in the batch. More "
            "efficient on GPU but very bad for TPU."
        },
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of training examples to this "
            "value if set."
        },
    )
    max_eval_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of evaluation examples to this "
            "value if set."
        },
    )
    max_predict_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of prediction examples to this "
            "value if set."
        },
    )
    num_beams: Optional[int] = field(
        default=None,
        metadata={
            "help": "Number of beams to use for evaluation. This argument will be passed to ``model.generate``, "
            "which is used during ``evaluate`` and ``predict``."
        },
    )
    ignore_pad_token_for_loss: bool = field(
        default=True,
        metadata={
            "help": "Whether to ignore the tokens corresponding to padded labels in the loss computation or not."
        },
    )
    source_prefix: Optional[str] = field(
        default="",
        metadata={
            "help": "A prefix to add before every source text (useful for T5 models)."
        },
    )

    forced_bos_token: Optional[str] = field(
        default=None,
        metadata={
            "help": "The token to force as the first generated token after the decoder_start_token_id."
            "Useful for multilingual models like mBART where the first generated token"
            "needs to be the target language token (Usually it is the target language token)"
        },
    )

    def __post_init__(self):
        if (
            self.dataset_name is None
            and self.train_file is None
            and self.validation_file is None
        ):
            raise ValueError(
                "Need either a dataset name or a training/validation file."
            )
        else:
            if self.train_file is not None:
                extension = self.train_file.split(".")[-1]
                assert extension in [
                    "csv",
                    "json",
                ], "`train_file` should be a csv or a json file."
            if self.validation_file is not None:
                extension = self.validation_file.split(".")[-1]
                assert extension in [
                    "csv",
                    "json",
                ], "`validation_file` should be a csv or a json file."
        if self.val_max_target_length is None:
            self.val_max_target_length = self.max_target_length

In [10]:
summarization_name_mapping = {
    "amazon_reviews_multi": ("review_body", "review_title"),
    "big_patent": ("description", "abstract"),
    "cnn_dailymail": ("article", "highlights"),
    "orange_sum": ("text", "summary"),
    "pn_summary": ("article", "summary"),
    "psc": ("extract_text", "summary_text"),
    "samsum": ("dialogue", "summary"),
    "thaisum": ("body", "summary"),
    "xglue": ("news_body", "news_title"),
    "xsum": ("document", "summary"),
    "wiki_summary": ("article", "highlights"),
}

In [5]:
test_dataset = load_dataset("xsum")

Downloading:   0%|          | 0.00/2.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/954 [00:00<?, ?B/s]

Using custom data configuration default


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset xsum downloaded and prepared to /home/cdsw/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
test_dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})

In [22]:
wnc_datasets_clean = load_from_disk("../data/processed/WNC_oneword")

In [23]:
wnc_datasets_clean

DatasetDict({
    dev: Dataset({
        features: ['rev_id', 'source_text', 'target_text'],
        num_rows: 671
    })
    test: Dataset({
        features: ['rev_id', 'source_text', 'target_text'],
        num_rows: 967
    })
    train: Dataset({
        features: ['rev_id', 'source_text', 'target_text'],
        num_rows: 52093
    })
})

In [ ]:


wnc_datasets_clean.pop(

In [24]:
wnc_datasets_clean["validation"] = wnc_datasets_clean.pop("dev")

In [25]:
wnc_datasets_clean

DatasetDict({
    test: Dataset({
        features: ['rev_id', 'source_text', 'target_text'],
        num_rows: 967
    })
    train: Dataset({
        features: ['rev_id', 'source_text', 'target_text'],
        num_rows: 52093
    })
    validation: Dataset({
        features: ['rev_id', 'source_text', 'target_text'],
        num_rows: 671
    })
})

In [5]:
def main():
    # See all possible arguments in src/transformers/training_args.py
    # or by passing the --help flag to this script.
    # We now keep distinct sets of args, for a cleaner separation of concerns.

    parser = HfArgumentParser((ModelArguments, DataTrainingArguments, Seq2SeqTrainingArguments))
    if len(sys.argv) == 2 and sys.argv[1].endswith(".json"):
        # If we pass only one argument to the script and it's the path to a json file,
        # let's parse it to get our arguments.
        model_args, data_args, training_args = parser.parse_json_file(json_file=os.path.abspath(sys.argv[1]))
    else:
        model_args, data_args, training_args = parser.parse_args_into_dataclasses()

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        handlers=[logging.StreamHandler(sys.stdout)],
    )
    log_level = training_args.get_process_log_level()
    logger.setLevel(log_level)
    datasets.utils.logging.set_verbosity(log_level)
    transformers.utils.logging.set_verbosity(log_level)
    transformers.utils.logging.enable_default_handler()
    transformers.utils.logging.enable_explicit_format()

    # Log on each process the small summary:
    logger.warning(
        f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
        + f"distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
    )
    logger.info(f"Training/evaluation parameters {training_args}")

    if data_args.source_prefix is None and model_args.model_name_or_path in [
        "t5-small",
        "t5-base",
        "t5-large",
        "t5-3b",
        "t5-11b",
    ]:
        logger.warning(
            "You're running a t5 model but didn't provide a source prefix, which is the expected, e.g. with "
            "`--source_prefix 'summarize: ' `"
        )

    # Detecting last checkpoint.
    last_checkpoint = None
    if os.path.isdir(training_args.output_dir) and training_args.do_train and not training_args.overwrite_output_dir:
        last_checkpoint = get_last_checkpoint(training_args.output_dir)
        if last_checkpoint is None and len(os.listdir(training_args.output_dir)) > 0:
            raise ValueError(
                f"Output directory ({training_args.output_dir}) already exists and is not empty. "
                "Use --overwrite_output_dir to overcome."
            )
        elif last_checkpoint is not None and training_args.resume_from_checkpoint is None:
            logger.info(
                f"Checkpoint detected, resuming training at {last_checkpoint}. To avoid this behavior, change "
                "the `--output_dir` or add `--overwrite_output_dir` to train from scratch."
            )

    # Set seed before initializing model.
    set_seed(training_args.seed)

    # Get the datasets: you can either provide your own CSV/JSON training and evaluation files (see below)
    # or just provide the name of one of the public datasets available on the hub at https://huggingface.co/datasets/
    # (the dataset will be downloaded automatically from the datasets Hub).
    #
    # For CSV/JSON files this script will use the first column for the full texts and the second column for the
    # summaries (unless you specify column names for this with the `text_column` and `summary_column` arguments).
    #
    # In distributed training, the load_dataset function guarantee that only one local process can concurrently
    # download the dataset.
    if data_args.dataset_name is not None:
        # Downloading and loading a dataset from the hub.
        raw_datasets = load_dataset(
            data_args.dataset_name, data_args.dataset_config_name, cache_dir=model_args.cache_dir
        )
    else:
        data_files = {}
        if data_args.train_file is not None:
            data_files["train"] = data_args.train_file
            extension = data_args.train_file.split(".")[-1]
        if data_args.validation_file is not None:
            data_files["validation"] = data_args.validation_file
            extension = data_args.validation_file.split(".")[-1]
        if data_args.test_file is not None:
            data_files["test"] = data_args.test_file
            extension = data_args.test_file.split(".")[-1]
        raw_datasets = load_dataset(extension, data_files=data_files, cache_dir=model_args.cache_dir)
    # See more about loading any type of standard or custom dataset (from files, python dict, pandas DataFrame, etc) at
    # https://huggingface.co/docs/datasets/loading_datasets.html.

    # Load pretrained model and tokenizer
    #
    # Distributed training:
    # The .from_pretrained methods guarantee that only one local process can concurrently
    # download model & vocab.
    config = AutoConfig.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
        revision=model_args.model_revision,
        use_auth_token=True if model_args.use_auth_token else None,
    )
    tokenizer = AutoTokenizer.from_pretrained(
        model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
        use_fast=model_args.use_fast_tokenizer,
        revision=model_args.model_revision,
        use_auth_token=True if model_args.use_auth_token else None,
    )
    model = AutoModelForSeq2SeqLM.from_pretrained(
        model_args.model_name_or_path,
        from_tf=bool(".ckpt" in model_args.model_name_or_path),
        config=config,
        cache_dir=model_args.cache_dir,
        revision=model_args.model_revision,
        use_auth_token=True if model_args.use_auth_token else None,
    )

    model.resize_token_embeddings(len(tokenizer))

    if model.config.decoder_start_token_id is None and isinstance(tokenizer, (MBartTokenizer, MBartTokenizerFast)):
        if isinstance(tokenizer, MBartTokenizer):
            model.config.decoder_start_token_id = tokenizer.lang_code_to_id[data_args.lang]
        else:
            model.config.decoder_start_token_id = tokenizer.convert_tokens_to_ids(data_args.lang)

    if model.config.decoder_start_token_id is None:
        raise ValueError("Make sure that `config.decoder_start_token_id` is correctly defined")

    if (
        hasattr(model.config, "max_position_embeddings")
        and model.config.max_position_embeddings < data_args.max_source_length
    ):
        if model_args.resize_position_embeddings is None:
            logger.warning(
                f"Increasing the model's number of position embedding vectors from {model.config.max_position_embeddings} "
                f"to {data_args.max_source_length}."
            )
            model.resize_position_embeddings(data_args.max_source_length)
        elif model_args.resize_position_embeddings:
            model.resize_position_embeddings(data_args.max_source_length)
        else:
            raise ValueError(
                f"`--max_source_length` is set to {data_args.max_source_length}, but the model only has {model.config.max_position_embeddings}"
                f" position encodings. Consider either reducing `--max_source_length` to {model.config.max_position_embeddings} or to automatically "
                "resize the model's position encodings by passing `--resize_position_embeddings`."
            )

    prefix = data_args.source_prefix if data_args.source_prefix is not None else ""

    # Preprocessing the datasets.
    # We need to tokenize inputs and targets.
    if training_args.do_train:
        column_names = raw_datasets["train"].column_names
    elif training_args.do_eval:
        column_names = raw_datasets["validation"].column_names
    elif training_args.do_predict:
        column_names = raw_datasets["test"].column_names
    else:
        logger.info("There is nothing to do. Please pass `do_train`, `do_eval` and/or `do_predict`.")
        return

    if isinstance(tokenizer, tuple(MULTILINGUAL_TOKENIZERS)):
        assert (
            data_args.lang is not None
        ), f"{tokenizer.__class__.__name__} is a multilingual tokenizer which requires --lang argument"

        tokenizer.src_lang = data_args.lang
        tokenizer.tgt_lang = data_args.lang

        # For multilingual translation models like mBART-50 and M2M100 we need to force the target language token
        # as the first generated token. We ask the user to explicitly provide this as --forced_bos_token argument.
        forced_bos_token_id = (
            tokenizer.lang_code_to_id[data_args.forced_bos_token] if data_args.forced_bos_token is not None else None
        )
        model.config.forced_bos_token_id = forced_bos_token_id

    # Get the column names for input/target.
    dataset_columns = summarization_name_mapping.get(data_args.dataset_name, None)
    if data_args.text_column is None:
        text_column = dataset_columns[0] if dataset_columns is not None else column_names[0]
    else:
        text_column = data_args.text_column
        if text_column not in column_names:
            raise ValueError(
                f"--text_column' value '{data_args.text_column}' needs to be one of: {', '.join(column_names)}"
            )
    if data_args.summary_column is None:
        summary_column = dataset_columns[1] if dataset_columns is not None else column_names[1]
    else:
        summary_column = data_args.summary_column
        if summary_column not in column_names:
            raise ValueError(
                f"--summary_column' value '{data_args.summary_column}' needs to be one of: {', '.join(column_names)}"
            )

    # Temporarily set max_target_length for training.
    max_target_length = data_args.max_target_length
    padding = "max_length" if data_args.pad_to_max_length else False

    if training_args.label_smoothing_factor > 0 and not hasattr(model, "prepare_decoder_input_ids_from_labels"):
        logger.warning(
            "label_smoothing is enabled but the `prepare_decoder_input_ids_from_labels` method is not defined for"
            f"`{model.__class__.__name__}`. This will lead to loss being calculated twice and will take up more memory"
        )

    def preprocess_function(examples):
        # remove pairs where at least one record is None

        inputs, targets = [], []
        for i in range(len(examples[text_column])):
            if examples[text_column][i] is not None and examples[summary_column][i] is not None:
                inputs.append(examples[text_column][i])
                targets.append(examples[summary_column][i])

        inputs = [prefix + inp for inp in inputs]
        model_inputs = tokenizer(inputs, max_length=data_args.max_source_length, padding=padding, truncation=True)

        # Setup the tokenizer for targets
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(targets, max_length=max_target_length, padding=padding, truncation=True)

        # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
        # padding in the loss.
        if padding == "max_length" and data_args.ignore_pad_token_for_loss:
            labels["input_ids"] = [
                [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
            ]

        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    if training_args.do_train:
        if "train" not in raw_datasets:
            raise ValueError("--do_train requires a train dataset")
        train_dataset = raw_datasets["train"]
        if data_args.max_train_samples is not None:
            train_dataset = train_dataset.select(range(data_args.max_train_samples))
        with training_args.main_process_first(desc="train dataset map pre-processing"):
            train_dataset = train_dataset.map(
                preprocess_function,
                batched=True,
                num_proc=data_args.preprocessing_num_workers,
                remove_columns=column_names,
                load_from_cache_file=not data_args.overwrite_cache,
                desc="Running tokenizer on train dataset",
            )

    if training_args.do_eval:
        max_target_length = data_args.val_max_target_length
        if "validation" not in raw_datasets:
            raise ValueError("--do_eval requires a validation dataset")
        eval_dataset = raw_datasets["validation"]
        if data_args.max_eval_samples is not None:
            eval_dataset = eval_dataset.select(range(data_args.max_eval_samples))
        with training_args.main_process_first(desc="validation dataset map pre-processing"):
            eval_dataset = eval_dataset.map(
                preprocess_function,
                batched=True,
                num_proc=data_args.preprocessing_num_workers,
                remove_columns=column_names,
                load_from_cache_file=not data_args.overwrite_cache,
                desc="Running tokenizer on validation dataset",
            )

    if training_args.do_predict:
        max_target_length = data_args.val_max_target_length
        if "test" not in raw_datasets:
            raise ValueError("--do_predict requires a test dataset")
        predict_dataset = raw_datasets["test"]
        if data_args.max_predict_samples is not None:
            predict_dataset = predict_dataset.select(range(data_args.max_predict_samples))
        with training_args.main_process_first(desc="prediction dataset map pre-processing"):
            predict_dataset = predict_dataset.map(
                preprocess_function,
                batched=True,
                num_proc=data_args.preprocessing_num_workers,
                remove_columns=column_names,
                load_from_cache_file=not data_args.overwrite_cache,
                desc="Running tokenizer on prediction dataset",
            )

    # Data collator
    label_pad_token_id = -100 if data_args.ignore_pad_token_for_loss else tokenizer.pad_token_id
    data_collator = DataCollatorForSeq2Seq(
        tokenizer,
        model=model,
        label_pad_token_id=label_pad_token_id,
        pad_to_multiple_of=8 if training_args.fp16 else None,
    )

    # Metric
    metric = load_metric("rouge")

    def postprocess_text(preds, labels):
        preds = [pred.strip() for pred in preds]
        labels = [label.strip() for label in labels]

        # rougeLSum expects newline after each sentence
        preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
        labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

        return preds, labels

    def compute_metrics(eval_preds):
        preds, labels = eval_preds
        if isinstance(preds, tuple):
            preds = preds[0]
        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
        if data_args.ignore_pad_token_for_loss:
            # Replace -100 in the labels as we can't decode them.
            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        # Some simple post-processing
        decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

        result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
        # Extract a few results from ROUGE
        result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

        prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
        result["gen_len"] = np.mean(prediction_lens)
        result = {k: round(v, 4) for k, v in result.items()}
        return result

    # Initialize our Trainer
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset if training_args.do_train else None,
        eval_dataset=eval_dataset if training_args.do_eval else None,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics if training_args.predict_with_generate else None,
    )

    # Training
    if training_args.do_train:
        checkpoint = None
        if training_args.resume_from_checkpoint is not None:
            checkpoint = training_args.resume_from_checkpoint
        elif last_checkpoint is not None:
            checkpoint = last_checkpoint
        train_result = trainer.train(resume_from_checkpoint=checkpoint)
        trainer.save_model()  # Saves the tokenizer too for easy upload

        metrics = train_result.metrics
        max_train_samples = (
            data_args.max_train_samples if data_args.max_train_samples is not None else len(train_dataset)
        )
        metrics["train_samples"] = min(max_train_samples, len(train_dataset))

        trainer.log_metrics("train", metrics)
        trainer.save_metrics("train", metrics)
        trainer.save_state()

    # Evaluation
    results = {}
    max_length = (
        training_args.generation_max_length
        if training_args.generation_max_length is not None
        else data_args.val_max_target_length
    )
    num_beams = data_args.num_beams if data_args.num_beams is not None else training_args.generation_num_beams
    if training_args.do_eval:
        logger.info("*** Evaluate ***")
        metrics = trainer.evaluate(max_length=max_length, num_beams=num_beams, metric_key_prefix="eval")
        max_eval_samples = data_args.max_eval_samples if data_args.max_eval_samples is not None else len(eval_dataset)
        metrics["eval_samples"] = min(max_eval_samples, len(eval_dataset))

        trainer.log_metrics("eval", metrics)
        trainer.save_metrics("eval", metrics)

    if training_args.do_predict:
        logger.info("*** Predict ***")

        predict_results = trainer.predict(
            predict_dataset, metric_key_prefix="predict", max_length=max_length, num_beams=num_beams
        )
        metrics = predict_results.metrics
        max_predict_samples = (
            data_args.max_predict_samples if data_args.max_predict_samples is not None else len(predict_dataset)
        )
        metrics["predict_samples"] = min(max_predict_samples, len(predict_dataset))

        trainer.log_metrics("predict", metrics)
        trainer.save_metrics("predict", metrics)

        if trainer.is_world_process_zero():
            if training_args.predict_with_generate:
                predictions = tokenizer.batch_decode(
                    predict_results.predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True
                )
                predictions = [pred.strip() for pred in predictions]
                output_prediction_file = os.path.join(training_args.output_dir, "generated_predictions.txt")
                with open(output_prediction_file, "w") as writer:
                    writer.write("\n".join(predictions))

    kwargs = {"finetuned_from": model_args.model_name_or_path, "tasks": "summarization"}
    if data_args.dataset_name is not None:
        kwargs["dataset_tags"] = data_args.dataset_name
        if data_args.dataset_config_name is not None:
            kwargs["dataset_args"] = data_args.dataset_config_name
            kwargs["dataset"] = f"{data_args.dataset_name} {data_args.dataset_config_name}"
        else:
            kwargs["dataset"] = data_args.dataset_name

    if data_args.lang is not None:
        kwargs["language"] = data_args.lang

    if training_args.push_to_hub:
        trainer.push_to_hub(**kwargs)
    else:
        trainer.create_model_card(**kwargs)

    return results


In [ ]:
load_metric(